In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=2*forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(np.log(rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(np.log(rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-3*forecastDay] & np.log
    ds = rawArrayDatas[0][:-3*forecastDay]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-3*forecastDay]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of 2*forecastDay:  rawArrayDatas[1][-3*forecastDay:-forecastDay]
    testY= rawArrayDatas[1][-3*forecastDay:-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
        listedLogPredict=test_predict[-1].tolist()
    return [np.exp(y) for y in listedLogPredict]

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm



In [2]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [26]:
columns=['ds','y']
df=pd.read_table('walMonth.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.1)
print(len(df), forecastDay)


33 3


In [27]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
rawArrayDatas

[['2010-02-01',
  '2010-03-01',
  '2010-04-01',
  '2010-05-01',
  '2010-06-01',
  '2010-07-01',
  '2010-08-01',
  '2010-09-01',
  '2010-10-01',
  '2010-11-01',
  '2010-12-01',
  '2011-01-01',
  '2011-02-01',
  '2011-03-01',
  '2011-04-01',
  '2011-05-01',
  '2011-06-01',
  '2011-07-01',
  '2011-08-01',
  '2011-09-01',
  '2011-10-01',
  '2011-11-01',
  '2011-12-01',
  '2012-01-01',
  '2012-02-01',
  '2012-03-01',
  '2012-04-01',
  '2012-05-01',
  '2012-06-01',
  '2012-07-01',
  '2012-08-01',
  '2012-09-01',
  '2012-10-01'],
 [32990.769999999997,
  22809.285,
  30103.352000000003,
  16673.537499999999,
  16685.174999999999,
  16383.002,
  16144.702499999999,
  17978.317500000001,
  26928.906000000003,
  23040.349999999999,
  34796.775999999998,
  17286.647499999999,
  31440.657500000001,
  20705.834999999999,
  33011.389999999999,
  17062.93,
  15744.6425,
  15771.246000000001,
  14765.487499999999,
  17551.281999999999,
  24701.7075,
  24634.377499999999,
  34902.413999999997,
  17551.3

In [30]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #7은 processId

[step: 0] loss: 0.4497160017490387
[step: 1] loss: 0.15955907106399536
[step: 2] loss: 0.030341386795043945
[step: 3] loss: 0.012750663794577122
[step: 4] loss: 0.04733975604176521
[step: 5] loss: 0.08296124637126923
[step: 6] loss: 0.09543914347887039
[step: 7] loss: 0.08449675887823105
[step: 8] loss: 0.06093190237879753
[step: 9] loss: 0.03567023575305939
[step: 10] loss: 0.015806075185537338
[step: 11] loss: 0.004462819546461105
[step: 12] loss: 0.0016636924119666219
[step: 13] loss: 0.005317428149282932
[step: 14] loss: 0.012218638323247433
[step: 15] loss: 0.01907535269856453
[step: 16] loss: 0.023439636453986168
[step: 17] loss: 0.024225562810897827
[step: 18] loss: 0.021643398329615593
[step: 19] loss: 0.016770483925938606
[step: 20] loss: 0.011046843603253365
[step: 21] loss: 0.0058437371626496315
[step: 22] loss: 0.002152955625206232
[step: 23] loss: 0.0004130933084525168
[step: 24] loss: 0.0004892863798886538
[step: 25] loss: 0.0018091389210894704
[step: 26] loss: 0.00361329

[step: 253] loss: 3.7696656818297924e-06
[step: 254] loss: 3.7653160234185634e-06
[step: 255] loss: 3.760921572393272e-06
[step: 256] loss: 3.756556907319464e-06
[step: 257] loss: 3.752242264454253e-06
[step: 258] loss: 3.7477593650692143e-06
[step: 259] loss: 3.743466095329495e-06
[step: 260] loss: 3.7390566376416245e-06
[step: 261] loss: 3.7347706438595196e-06
[step: 262] loss: 3.7304405395843787e-06
[step: 263] loss: 3.7259942473610863e-06
[step: 264] loss: 3.721669600054156e-06
[step: 265] loss: 3.717224444699241e-06
[step: 266] loss: 3.712914349307539e-06
[step: 267] loss: 3.7085885651322315e-06
[step: 268] loss: 3.7041716041130712e-06
[step: 269] loss: 3.699845365190413e-06
[step: 270] loss: 3.695514806167921e-06
[step: 271] loss: 3.691088750201743e-06
[step: 272] loss: 3.6867311337118736e-06
[step: 273] loss: 3.682408987515373e-06
[step: 274] loss: 3.678039320220705e-06
[step: 275] loss: 3.6735948469868163e-06
[step: 276] loss: 3.6692540561489295e-06
[step: 277] loss: 3.66493122

[step: 503] loss: 2.762982376225409e-06
[step: 504] loss: 2.759520157269435e-06
[step: 505] loss: 2.7560365651879692e-06
[step: 506] loss: 2.752615728240926e-06
[step: 507] loss: 2.7491482796904165e-06
[step: 508] loss: 2.745711981333443e-06
[step: 509] loss: 2.742332981142681e-06
[step: 510] loss: 2.7388671242079e-06
[step: 511] loss: 2.735485622906708e-06
[step: 512] loss: 2.73201112577226e-06
[step: 513] loss: 2.7285650503472425e-06
[step: 514] loss: 2.7251614937995328e-06
[step: 515] loss: 2.721780447245692e-06
[step: 516] loss: 2.718407586144167e-06
[step: 517] loss: 2.714986521823448e-06
[step: 518] loss: 2.7116548153571784e-06
[step: 519] loss: 2.7081748612545198e-06
[step: 520] loss: 2.7048263291362673e-06
[step: 521] loss: 2.7015362320526037e-06
[step: 522] loss: 2.698130629141815e-06
[step: 523] loss: 2.6946918296744116e-06
[step: 524] loss: 2.6913712645182386e-06
[step: 525] loss: 2.6879711185756605e-06
[step: 526] loss: 2.6845859792956617e-06
[step: 527] loss: 2.68131566372

[step: 754] loss: 2.0558268261083867e-06
[step: 755] loss: 2.053641537713702e-06
[step: 756] loss: 2.051441470030113e-06
[step: 757] loss: 2.049182285190909e-06
[step: 758] loss: 2.047068164756638e-06
[step: 759] loss: 2.0447921542654512e-06
[step: 760] loss: 2.0426130049600033e-06
[step: 761] loss: 2.040389063040493e-06
[step: 762] loss: 2.0382049115141854e-06
[step: 763] loss: 2.036031673924299e-06
[step: 764] loss: 2.0338486592663685e-06
[step: 765] loss: 2.0316488189564552e-06
[step: 766] loss: 2.029441930062603e-06
[step: 767] loss: 2.0272593701520236e-06
[step: 768] loss: 2.0251006844773656e-06
[step: 769] loss: 2.022934040724067e-06
[step: 770] loss: 2.020685542447609e-06
[step: 771] loss: 2.0186153051326983e-06
[step: 772] loss: 2.0164568468317157e-06
[step: 773] loss: 2.014274286921136e-06
[step: 774] loss: 2.0121967736486113e-06
[step: 775] loss: 2.009911213463056e-06
[step: 776] loss: 2.0078857687622076e-06
[step: 777] loss: 2.00573504116619e-06
[step: 778] loss: 2.003624558

testY is:  [33670.824999999997, 22936.107999999997, 31400.029999999999, 17698.244999999999, 16729.308000000001, 16617.197499999998]


LSTM testforecast : [20659.65165046417, 21041.465883492256, 42424.21822808135, 25271.560698865589, 32790.586842107092, 16207.489377113679] 
@@@@@LSTM rmse:  10082.1680558
Bayseian testforecast : [20953.691612653518, 18360.313809394112, 36190.737818519941, 17831.411781722676, 15210.407792773669, 15590.762041885999] 
@@@@@Bayseian rmse:  5901.87281646


Bayseian WON!!!!!!
[step: 0] loss: 0.1272420734167099
[step: 1] loss: 0.008132096379995346
[step: 2] loss: 0.04031354933977127
[step: 3] loss: 0.06276451796293259
[step: 4] loss: 0.03878691792488098
[step: 5] loss: 0.009004450403153896
[step: 6] loss: 0.0003044874465558678
[step: 7] loss: 0.011315764859318733
[step: 8] loss: 0.02378539741039276
[step: 9] loss: 0.02541215717792511
[step: 10] loss: 0.0175168476998806
[step: 11] loss: 0.007338566705584526
[step: 12] loss: 0.0010996346827596426
[step: 13] loss:

[step: 235] loss: 1.3377452887652908e-06
[step: 236] loss: 1.335459046458709e-06
[step: 237] loss: 1.3332106618690887e-06
[step: 238] loss: 1.3309755786394817e-06
[step: 239] loss: 1.3286958164826501e-06
[step: 240] loss: 1.3265098459669389e-06
[step: 241] loss: 1.3242063232610235e-06
[step: 242] loss: 1.3220243317846325e-06
[step: 243] loss: 1.319785837949894e-06
[step: 244] loss: 1.3175683761801338e-06
[step: 245] loss: 1.3153016880096402e-06
[step: 246] loss: 1.31310218876024e-06
[step: 247] loss: 1.3108874554745853e-06
[step: 248] loss: 1.3087405932310503e-06
[step: 249] loss: 1.3065465509498608e-06
[step: 250] loss: 1.3043450053373817e-06
[step: 251] loss: 1.3021060567552922e-06
[step: 252] loss: 1.299901327911357e-06
[step: 253] loss: 1.297789140153327e-06
[step: 254] loss: 1.2956551245224546e-06
[step: 255] loss: 1.29344437027612e-06
[step: 256] loss: 1.29130773984798e-06
[step: 257] loss: 1.2891648566437652e-06
[step: 258] loss: 1.287046643483336e-06
[step: 259] loss: 1.2848204

[step: 470] loss: 9.535054914522334e-07
[step: 471] loss: 9.524329698251677e-07
[step: 472] loss: 9.514025123280589e-07
[step: 473] loss: 9.503562523605069e-07
[step: 474] loss: 9.492865160609654e-07
[step: 475] loss: 9.48300851177919e-07
[step: 476] loss: 9.472057058701466e-07
[step: 477] loss: 9.462151524530782e-07
[step: 478] loss: 9.452022595723975e-07
[step: 479] loss: 9.442182999919169e-07
[step: 480] loss: 9.431236094314954e-07
[step: 481] loss: 9.421057143299549e-07
[step: 482] loss: 9.411145924786979e-07
[step: 483] loss: 9.401549618814897e-07
[step: 484] loss: 9.391503112965438e-07
[step: 485] loss: 9.381976724398555e-07
[step: 486] loss: 9.371467513119569e-07
[step: 487] loss: 9.361448860545352e-07
[step: 488] loss: 9.351564926873834e-07
[step: 489] loss: 9.341767963633174e-07
[step: 490] loss: 9.332076160717406e-07
[step: 491] loss: 9.322579330728331e-07
[step: 492] loss: 9.312944371231424e-07
[step: 493] loss: 9.302763714913453e-07
[step: 494] loss: 9.293485163652804e-07
[

[step: 713] loss: 7.904162657723646e-07
[step: 714] loss: 7.905067036517721e-07
[step: 715] loss: 7.907909207460762e-07
[step: 716] loss: 7.915000423963647e-07
[step: 717] loss: 7.927986871436588e-07
[step: 718] loss: 7.951230145408772e-07
[step: 719] loss: 7.991310440047528e-07
[step: 720] loss: 8.059469109866768e-07
[step: 721] loss: 8.175666152965277e-07
[step: 722] loss: 8.373414743800822e-07
[step: 723] loss: 8.712826797818707e-07
[step: 724] loss: 9.300267151957087e-07
[step: 725] loss: 1.03285049135593e-06
[step: 726] loss: 1.214339249600016e-06
[step: 727] loss: 1.5383054687845288e-06
[step: 728] loss: 2.1218320398475043e-06
[step: 729] loss: 3.184989054716425e-06
[step: 730] loss: 5.136425897944719e-06
[step: 731] loss: 8.75523346621776e-06
[step: 732] loss: 1.5482182789128274e-05
[step: 733] loss: 2.805952499329578e-05
[step: 734] loss: 5.129606506670825e-05
[step: 735] loss: 9.368864994030446e-05
[step: 736] loss: 0.0001666435127845034
[step: 737] loss: 0.0002821369271259755

[step: 954] loss: 7.239666501845932e-07
[step: 955] loss: 7.238097055051185e-07
[step: 956] loss: 7.236206442939874e-07
[step: 957] loss: 7.234676218104141e-07
[step: 958] loss: 7.232882239804894e-07
[step: 959] loss: 7.2313400778512e-07
[step: 960] loss: 7.229768357319699e-07
[step: 961] loss: 7.228516665236384e-07
[step: 962] loss: 7.226342972899147e-07
[step: 963] loss: 7.224500677693868e-07
[step: 964] loss: 7.222994327094057e-07
[step: 965] loss: 7.221578357530234e-07
[step: 966] loss: 7.219668987090699e-07
[step: 967] loss: 7.217853408292285e-07
[step: 968] loss: 7.216675612653489e-07
[step: 969] loss: 7.214848096737114e-07
[step: 970] loss: 7.213205890366226e-07
[step: 971] loss: 7.21150115623459e-07
[step: 972] loss: 7.210161356852041e-07
[step: 973] loss: 7.208779493339534e-07
[step: 974] loss: 7.207558496702404e-07
[step: 975] loss: 7.205586598502123e-07
[step: 976] loss: 7.205567840173899e-07
[step: 977] loss: 7.205549081845675e-07
[step: 978] loss: 7.206573968687735e-07
[st

In [31]:
len(answer)

3

In [32]:
answer

[17001.131946273275, 22276.375241595382, 29180.097344759895]

In [33]:
len(rawArrayDatas[1])

33